# Homework 4

## Question 1

In [3]:
!uv pip show scikit-learn

Name: scikit-learn
Version: 1.7.0
Location: /Users/patrick/Projects/mlops_zoomcamp/04-deployment/.venv/lib/python3.12/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by:


In [7]:
!python --version

Python 3.12.10


In [27]:
import pickle
import pandas as pd
from pathlib import Path

In [11]:
with open('artifacts/model.pkl', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [12]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [14]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [15]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [20]:
print(f'The standard dev of the predictions is: {y_pred.std()}')

The standard dev of the predictions is: 6.247484442678229


## Question 2

In [41]:
year = 2023
month = 3

output_folder = Path('data')
output_folder.mkdir(parents=True, exist_ok=True)

output_file = output_folder / f'{year:04d}-{month:02d}_predictions.parquet' 

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df['predictions'] = y_pred

df_result = df[['ride_id', 'predictions']]

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [43]:
!ls -lh data

total 134072
-rw-r--r--  1 patrick  staff    65M 12 jun. 22:54 2023-03_predictions.parquet


## Question 3

In [45]:
!jupyter nbconvert --to script pipeline.ipynb

[NbConvertApp] Converting notebook pipeline.ipynb to script
[NbConvertApp] Writing 1576 bytes to pipeline.py


## Question 4

In [2]:
!uv export -o requirements.txt

Resolved 111 packages in 0.60ms
# This file was autogenerated by uv via the following command:
#    uv export -o requirements.txt
anyio==4.9.0 \
    --hash=sha256:673c0c244e15788651a4ff38710fea9675823028a6f08a5eda409e0c9840a028 \
    --hash=sha256:9f76d541cad6e36af7beb62e978876f3b41e3e04f2c1fbf0884604c0a9c4d93c
    # via
    #   httpx
    #   jupyter-server
appnope==0.1.4 ; sys_platform == 'darwin' \
    --hash=sha256:1de3860566df9caf38f01f86f65e0e13e379af54f9e4bee1e66b48f2efffd1ee \
    --hash=sha256:502575ee11cd7a28c0205f379b525beefebab9d161b7c964670864014ed7213c
    # via ipykernel
argon2-cffi==25.1.0 \
    --hash=sha256:694ae5cc8a42f4c4e2bf2ca0e64e51e23a040c6a517a85074683d3959e1346c1 \
    --hash=sha256:fdc8b074db390fccb6eb4a3604ae7231f219aa669a2652e0f20e16ba513d5741
    # via jupyter-server
argon2-cffi-bindings==21.2.0 \
    --hash=sha256:58ed19212051f49a523abb1dbe954337dc82d947fb6e5a0da60f7c8471a8476c \
    --hash=sha256:603ca0aba86b1349b147cab91ae970c63118a0f30444d4bc80355937c95

In [3]:
!cat requirements.txt

# This file was autogenerated by uv via the following command:
#    uv export -o requirements.txt
anyio==4.9.0 \
    --hash=sha256:673c0c244e15788651a4ff38710fea9675823028a6f08a5eda409e0c9840a028 \
    --hash=sha256:9f76d541cad6e36af7beb62e978876f3b41e3e04f2c1fbf0884604c0a9c4d93c
    # via
    #   httpx
    #   jupyter-server
appnope==0.1.4 ; sys_platform == 'darwin' \
    --hash=sha256:1de3860566df9caf38f01f86f65e0e13e379af54f9e4bee1e66b48f2efffd1ee \
    --hash=sha256:502575ee11cd7a28c0205f379b525beefebab9d161b7c964670864014ed7213c
    # via ipykernel
argon2-cffi==25.1.0 \
    --hash=sha256:694ae5cc8a42f4c4e2bf2ca0e64e51e23a040c6a517a85074683d3959e1346c1 \
    --hash=sha256:fdc8b074db390fccb6eb4a3604ae7231f219aa669a2652e0f20e16ba513d5741
    # via jupyter-server
argon2-cffi-bindings==21.2.0 \
    --hash=sha256:58ed19212051f49a523abb1dbe954337dc82d947fb6e5a0da60f7c8471a8476c \
    --hash=sha256:603ca0aba86b1349b147cab91ae970c63118a0f30444d4bc80355937c950c082 \
    --hash=sha256:8cd69c

`scikit-learn==1.7.0 \
    --hash=sha256:014e07a23fe02e65f9392898143c542a50b6001dbe89cb867e19688e468d049b \`

## Question 5